## Word Prediction using Markov Model

In [1]:
#import libraries
import string
import numpy as np

In [9]:
#import dataset
data_file = '../data/alllines.txt'

##### Preprocessing of the data

In [11]:
def cleanup_data(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

In [13]:
def create_dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [14]:
def probability_dict(list_data):
    probability_dict = {}
    given_list_length = len(list_data)
    for item in list_data:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict

In [15]:
#List to hold the initial states and transition states
initial_word = {}
second_word = {}
transitions = {}

### Training model

##### The training of the Markov model can be divided into the following stages -
1. Tokenisation
2. Building the state pairs
3. Determining the probability distribution

In [16]:
# Trains a Markov model based on the data in data_file
def train_markov_model():
    for line in open(data_file):
        tokens = cleanup_data(line.rstrip().lower()).split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:
                initial_word[token] = initial_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == tokens_length - 1:
                    create_dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    create_dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    create_dict(transitions, (prev_prev_token, prev_token), token)
    
    # Normalize the distributions
    initial_word_total = sum(initial_word.values())
    for key, value in initial_word.items():
        initial_word[key] = value / initial_word_total
        
    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = probability_dict(next_word_list)
        
    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = probability_dict(next_word_list)
    
    print('Training successful.')

In [17]:
train_markov_model()

Training successful.


### 1. Generating new text from the text corpus

Once we have completed the training, we will have the initial word distribution, second-word distribution and the state transition distributions. Next to generate a text corpus all we need is to write a function to sample out from the above-created distributions.

In [18]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [65]:
#Fixing our generated text to length 15
number_of_sentences = 15

In [20]:
# Function to generate sample text
def generate_text():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [21]:
generate_text()

he was of more
and fit man for brother men
he hath had no hands to the general wreck and but that you fear
persuade my loving proteus
could speak for me
you found it beggars any man
if you be bold to ask at what i desire her name
for here have i not cause to sigh as if the first edward the third yonder they lie the poor gentleman born
knowing that thou shalt know i know his means are no fool
why she hath beheld the doing
which daily she was not inclined that way
and prove untrue
within this bosom
they wound
pray god i pray you sir


### 2. Performing text prediction given a sequence of words

In [62]:
def text_prediction(text):
        text = cleanup_data(text.lower()).split()
        # Initial word
        word0 = text[0]
        # Second word
        if len(text) == 1:
            word1 = sample_word(second_word[word0])
            text.append(word1)
        else:
            word1 = text[1]
        # Subsequent words untill END
        while True:
            word2 = max(transitions[(word0, word1)], key=transitions[(word0, word1)].get)
            if word2 == 'END':
                break
            text.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(text))

In [63]:
#Testing
text_prediction("Whose arms")

whose arms were moulded in their own


In [64]:
text_prediction("Of hostile")

of hostile paces those opposed eyes


In [ ]:
##https://en.wikipedia.org/wiki/Hidden_Markov_model
##reference https://medium.com/ymedialabs-innovation/next-word-prediction-using-markov-model-570fc0475f96